# 1. Create FRCNN Model

Title: Faster-RCNN

Author: [Blaine Perry]

Date created: 2021/08/03

Last modified: 2021/08/03

Description: Implementation of Faster-RCNN.

## Train the model

# Start by creating the workspace

In [13]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt

In [14]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.32.0


In [15]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: geointerns_workspace
Azure region: usgovarizona
Subscription id: 8dab0d11-5037-44d3-9b24-ed18e6c0845e
Resource group: iso_lrpf


## Create an experiment

In [16]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='keras-FRCNN-ZSL')

## Create or Attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. Here we wil create `AmlCompute` as your training compute resource.


In [17]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "four-gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC24', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 1, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-08-23T21:33:35.495000+00:00', 'errors': None, 'creationTime': '2021-08-11T15:46:47.330021+00:00', 'modifiedTime': '2021-08-11T15:47:12.751770+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC24'}


## Create an environment

Define a conda environment YAML file with your training script dependencies, which include TensorFlow, Keras and matplotlib, and create an Azure ML environment.

In [18]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.8.8
- pip:
  - azureml-defaults
  - tensorflow-gpu==2.5
  - matplotlib
  - pandas
  - tqdm
  - tensorflow_hub
  - tensorflow_text
  - tensorflow_addons

Overwriting conda_dependencies.yml


In [19]:
from azureml.core import Environment

keras_ZSL_env = Environment.from_conda_specification(name = 'keras-ZSL-env', file_path = './conda_dependencies.yml')
# load the string from a file.
keras_ZSL_env.docker.base_image = None
keras_ZSL_env.docker.base_dockerfile = "./docker-FRCNN.txt"


keras_FRCNN_env = Environment.from_conda_specification(name = 'keras-FRCNN-env', file_path = './conda_dependencies.yml')
# load the string from a file.
keras_FRCNN_env.docker.base_image = None
keras_FRCNN_env.docker.base_dockerfile = "./docker-FRCNN.txt"


## Configure the training job

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on.

Note that we are specifying a DatasetConsumptionConfig for our FileDataset as an argument to the training script. Azure ML will resolve this DatasetConsumptionConfig to the mount-point of the backing datastore, which we access from the training script.

In [26]:
from azureml.core import Dataset
dataset = Dataset.get_by_name(ws, 'xview_tfrecords')
#dataset = Dataset.get_by_name(ws, 'xview_small_sample')

# list the files referenced by the dataset
dataset.to_path()


['/xview_attribute_dict_text.pickle',
 '/xview_class_labels.txt',
 '/xview_test_sample.record',
 '/xview_train_sample.record']

Train ZSL

In [29]:
from azureml.core import ScriptRunConfig

args = ['--data-folder', dataset.as_named_input('xview_tfrecords').as_mount(),
        #'--input-weight-path', None,
        '--num-epochs', 30,
        '--model-type', 'ZSL']

src = ScriptRunConfig(source_directory='./ZSL',
                      script='train_ZSL.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=keras_ZSL_env)


run = exp.submit(src)

Train FRCNN

In [22]:
from azureml.core import ScriptRunConfig

args = ['--data-folder', dataset.as_named_input('xview_tfrecords').as_mount(),
        #'--input-weight-path', 'path',
        '--num-epochs', 30,
        '--model-type', 'FRCNN']

src = ScriptRunConfig(source_directory='./ZSL',
                      script='train_ZSL.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=keras_FRCNN_env)


run = exp.submit(src)